In [ ]:
!pip install mne

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

import mne
from mne.datasets import sample

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import SimpleRNN
from keras.losses import CategoricalCrossentropy
from keras.losses import BinaryCrossentropy
from keras.metrics import CategoricalAccuracy
from keras.metrics import BinaryAccuracy
from matplotlib import pyplot as plt
import random
from sklearn.model_selection import train_test_split


from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_left_ear = np.load('/content/drive/MyDrive/data/ROI/Left Auditory.npy')
data_right_ear = np.load('/content/drive/MyDrive/data/ROI/Right Auditory.npy')
data_right_visual = np.load('/content/drive/MyDrive/data/ROI/Right visual.npy')
data_left_visual = np.load('/content/drive/MyDrive/data/ROI/Left visual.npy')

In [ ]:
x = np.load('/content/drive/MyDrive/data/ROI/X_train.npy')
y = np.load('/content/drive/MyDrive/data/ROI/Y_train.npy')

In [ ]:
x_test = np.load('/content/drive/MyDrive/data/ROI/X_test.npy')
y_test = np.load('/content/drive/MyDrive/data/ROI/Y_test.npy')

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
def z(train_data, data_ref):
  data = train_data.copy()
  buf = np.resize(data_ref,(data_ref.shape[0]*data_ref.shape[1],data_ref.shape[2]))
  mean = np.mean(buf,axis = 0)
  std = np.std(buf, axis = 0)
  for i in range(data_ref.shape[2]):
    data[:,:,i] = (data[:,:,i] - mean[i])/std[i]
  return data

In [ ]:
train_data_norm = z(x,x)

In [ ]:
train_data, val_data, train_labels, val_labels = train_test_split(train_data_norm, y, test_size = 0.1,random_state = 0)

In [ ]:
model = Sequential()
model.add(SimpleRNN(261, return_sequences=True, activation="sigmoid"))
model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
callbacks = [ModelCheckpoint("model.h5", monitor='val_loss', save_best_only=True),
             EarlyStopping(monitor='loss', patience=15)]

In [ ]:
history = model.fit(train_data,train_labels,batch_size = 512, epochs = 100, validation_data=(val_data , val_labels), shuffle = True, callbacks=callbacks)

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('Categorical BinaryEntropy')
plt.xlabel('epoches')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
test_data_norm = z(x_test,x)

In [ ]:
y_pred= model.predict(test_data_norm)

164/164 [==============================] - 1s 3ms/step


In [ ]:
y_pred_thresh = np.zeros((5220,17,261))
for i in range(17):
  for j in range(261):
    if y_pred[0,i,j] >= 0.4:
      y_pred_thresh[0,i,j] = 1

In [ ]:
tn, fp, fn, tp,k = 0,0,0,0,0
for i in range(y_test.shape[0]):
  for j in range(y_test.shape[1]):
    buf= confusion_matrix(y_test[i][j], y_pred_thresh[i][j], labels = [0, 1]).ravel()
    tn+=buf[0]
    fp+=buf[1]
    fn+=buf[2]
    tp+=buf[3]
    k+=1

tp = tp/k
fn = fn/k
fp = fp/k

recall = tp/(tp+fn)
precision = tp/(tp+fp)
f1 = tp/(tp+(fp+fn)/2)

print(f'tn = {tn}, fp = {fp}, fn = {fn}, tp = {tp}')
print(f'Recall = {recall}')
print(f'Precision = {precision}')
print(f'F1 = {f1}')

In [ ]:
data_right_visual.shape

In [ ]:
buf = data_right_visual.T
buf = np.reshape(buf,(1,9,305))
real_norm = z(buf, x)
add = np.zeros((8,305))
buf = np.vstack((real_norm[0],add))
real_norm = np.reshape(buf,(1,17,305))

In [ ]:
real_pred= model.predict(real_norm)

1/1 [==============================] - 0s 238ms/step


In [ ]:
data_path = sample.data_path()
subjects_dir = data_path / 'subjects'
subject = 'sample'

labels = mne.read_labels_from_annot(
    subject = 'sample', parc = 'aparc.a2009s', hemi = "lh", subjects_dir=subjects_dir
)

new = []

for i in range(len(labels)):
    if len(labels[i].vertices) <= 1200:
        new.append(labels[i].split(parts = 3, subject = 'sample', subjects_dir = subjects_dir))
    elif len(labels[i].vertices) >= 4300:
        new.append(labels[i].split(parts = 7, subject = 'sample', subjects_dir = subjects_dir))
    elif len(labels[i].vertices) <= 3100 and len(labels[i].vertices) >= 1100:
        new.append(labels[i].split(parts = 3, subject = 'sample', subjects_dir = subjects_dir))
    else:
        new.append(labels[i].split(parts = 5, subject = 'sample', subjects_dir = subjects_dir))


new_labels = []
for i in range(len(new)):
    for j in range(len(new[i])):
        new_labels.append(new[i][j])


label_names = [label.name for label in new_labels]
n_labels = len(label_names)

Using default location ~/mne_data for sample...
Creating ~/mne_data


100%|██████████████████████████████████████| 1.65G/1.65G [00:00<00:00, 149GB/s]
Untarring contents of '/root/mne_data/MNE-sample-data-processed.tar.gz' to '/root/mne_data'


Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Download complete in 01m36s (1576.2 MB)
Reading labels from parcellation...
   read 75 labels from /root/mne_data/MNE-sample-data/subjects/sample/label/lh.aparc.a2009s.annot


In [ ]:
pred_regions= np.zeros((1,17,261))

In [ ]:
for i in range(17):
  for j in range(261):
    if real_pred[0,i,j] >= 0.4:
      pred_regions[0,i,j] = 1

In [ ]:
for i in range(17):
  print(np.where(pred_regions[0][i] == 1)[0])

[]
[63]
[ 16  63 171]
[16 63]
[16 27 63]
[ 8 16 27 28 33 35]
[  8  28  33  35 171]
[ 33  35 171]
[33]
[]
[]
[]
[]
[]
[]
[]
[]


In [ ]:
for i in range(17):
  activ_labels = np.where(pred_regions[0][i] == 1)[0]
  name_activ_labs = np.array(label_names)[activ_labels]
  print(name_activ_labs)

[]
['G_oc-temp_lat-fusifor_div2-lh']
['G_and_S_occipital_inf_div2-lh' 'G_oc-temp_lat-fusifor_div2-lh'
 'S_collat_transv_ant_div3-lh']
['G_and_S_occipital_inf_div2-lh' 'G_oc-temp_lat-fusifor_div2-lh']
['G_and_S_occipital_inf_div2-lh' 'G_cingul-Post-dorsal_div1-lh'
 'G_oc-temp_lat-fusifor_div2-lh']
['G_and_S_cingul-Mid-Ant_div3-lh' 'G_and_S_occipital_inf_div2-lh'
 'G_cingul-Post-dorsal_div1-lh' 'G_cingul-Post-dorsal_div2-lh'
 'G_cuneus_div1-lh' 'G_cuneus_div3-lh']
['G_and_S_cingul-Mid-Ant_div3-lh' 'G_cingul-Post-dorsal_div2-lh'
 'G_cuneus_div1-lh' 'G_cuneus_div3-lh' 'S_collat_transv_ant_div3-lh']
['G_cuneus_div1-lh' 'G_cuneus_div3-lh' 'S_collat_transv_ant_div3-lh']
['G_cuneus_div1-lh']
[]
[]
[]
[]
[]
[]
[]
[]
